# Getting starting

## Introduction
This is a colab notebook. It is how yo will do your analysis for this class. It requires a google account and is an example of an ipython notebook interface. 

This is an example of a text block. To add text blocks, simply click the "+ Text" button on the bar above the notebook. 

The code is marked up in a markup language called "Markdown". You can find a guide to markdown syntax here: [https://www.markdownguide.org/cheat-sheet/](https://www.markdownguide.org/cheat-sheet/).

Also, the [colab documentation](https://colab.research.google.com/notebooks/markdown_guide.ipynb) is useful.

Markdown should take you only a few moments to learn. It is as easy as a markup langauge can be made.



# Imports
The first thing we'll try is loading some data and plotting it. To do this, we'll need some packages. Let's load up pandas, a package for data management, and matplotlib. The python command for this is `import`.


In [0]:
import pandas as pd
import matplotlib as mpl


If you want to mount your google drive, do this.

In [0]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


To execute unix commands, do `!command`. For example, here I'm listing out the contents of my google drive

In [8]:
!ls drive/

'My Drive'  'Shared drives'


Let's download the data. Or you could upload the data to google drive. 

In [7]:
!wget https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt

--2019-08-01 19:13:35--  https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60977 (60K) [text/plain]
Saving to: ‘kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt.1’

kirby127a_3_1_ax_28 100%[===================>]  59.55K  --.-KB/s    in 0.03s   

2019-08-01 19:13:35 (2.32 MB/s) - ‘kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt.1’ saved [60977/60977]

drive
kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt.1
sample_data


In [25]:
dataset = pd.read_csv("/content/kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt",
                      skiprows = 9, 
                      nrows = 8,
                      delimiter = " ")

ParserError: ignored

# New Section

In [24]:
f=open('/content/kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt')
lines=f.readlines()[10 : 18]
f.close
f2 = open("type1level1.txt", "w")

f2.close

TypeError: ignored